# **저시력자를 위한 원화 화폐 분류**
---
- 본 과제는 UltraLytics YOLO v5 모델 사용을 권장합니다.
    - 본 파일의 목차는 UltraLytics YOLO v5에 맞게 작성되어 있습니다.
    - 다른 모델을 찾아서 사용하셔도 좋습니다.
    - 산출물이 잘 나오면 됩니다 : )
---

## 0.미션
---
- **과제 수행 목표**
    - 본 과제는 Object Detection 문제입니다.
    - Object Detection 문제로 접근하기 위해 **데이터셋 전처리**를 하셔야 합니다.
    - 데이터셋 : money_dataset.zip
        1. 데이터셋은 압축 파일로 제공됩니다.
        2. 압축 파일 안에는 화폐마다 폴더가 개별적으로 존재합니다.
        3. 폴더 안에는 화폐 이미지와 화폐 정보가 담긴 json 파일이 있습니다.
    - 여러분이 직접 촬영한 화폐 사진들을 탐지 과정에서 이용 해보세요.
    - 이미지에 화폐 하나만 나오게 촬영하는 것은 지양해주세요.
    - 다양한 방법으로 화폐를 촬영하고 결과를 확인해보세요.
        - ex 1) 화폐의 모든 종류를 한 이미지에 나오게 촬영
        - ex 2) 여러 화폐를 겹치게 하여 촬영
---
- **Key Point**
    1. 모델에 맞는 폴더 구조 확인
    2. 이미지 축소 비율에 맞춰 좌표값 변경
        - 좌표를 이미지 리사이즈한 비율로 변경
    3. 모델에 맞는 정보 추출/형식 변경
        - json 파일에서 정보 추출 및 모델 형식에 맞게 변경
    4. 화폐당 하나의 클래스로 변경
        - 총 8개 클래스
    5. 모델 선택 필요
---

## 1.환경설정

### (1) 구글 드라이브 연동, 데이터 다운로드
---
- 아래의 코드 셀을 반드시 실행시켜야 합니다.
---

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!pip install gdown

### (2) 데이터셋 불러오기
---
- **세부요구사항**
    - 데이터셋 파일의 압축을 해제하세요.
---
- 예제 코드에서는 zipfile 모듈을 이용하였습니다.
    - [zipfile document](https://docs.python.org/3/library/zipfile.html#zipfile-objects)
    - 해당 모듈 이외에 자신이 잘 알고 있는 방법을 사용해도 됩니다.
---

In [ ]:
import zipfile, gdown,os
url ="https://drive.google.com/file/d/1k1tXDK35s6BsMTPGWSl5GVGNoPfC898X/view?usp=drive_link"
file_name = "money_dataset.zip"
output = "/content/drive/MyDrive/" + file_name # 변경 가능
if not os.path.exists(output):
    gdown.download(url=url, output=output, quiet=False, fuzzy=True)

In [ ]:
# 데이터셋 압축 파일 경로 : 유저별로 상이할 수 있음
money_data = zipfile.ZipFile(output)

In [ ]:
# 데이터셋 압축 해제
money_data.extractall('/content/Dataset/')

## 2.데이터 전처리

### (1) 폴더 구조 생성 및 파일 이동
---
- **세부요구사항**
    -  모델에서 요구하는 폴더 구조를 만들어야 합니다.
        - Hint : Image와 Label을 구분하는 폴더를 만들어 주세요
---
- 예제 코드에서는 glob, shutil 모듈을 이용하였습니다.
    - [glob document](https://docs.python.org/3/library/glob.html) | [shutil document](https://docs.python.org/3/library/shutil.html)
    - 해당 모듈 이외에 자신이 잘 알고 있는 방법을 사용해도 됩니다.
---

In [ ]:
# 1.폴더 구조 만들기
!mkdir /content/Dataset/images;
!mkdir /content/Dataset/images/train; mkdir /content/Dataset/images/val

!mkdir /content/Dataset/labels;
!mkdir /content/Dataset/labels/train; mkdir /content/Dataset/labels/val

In [ ]:
import glob, shutil

In [ ]:
# 2. Dataset metadata 입력
won_list = ['10', '50', '100', '500', '1000', '5000', '10000', '50000']
data_path = '/content/Dataset/'

---
- 데이터를 Training set | Validation set으로 분할하세요.
    - 예시 : Training과 Validation은 8:2로 분리
- Hint : 이미지 데이터는 /images에, JSON 데이터는 /labels에 넣어주세요
    - 예시 : /dataset/images/train, /dataset/labels/train
    - 예제 코드에서는 glob, shutil 모듈을 이용하였습니다.
    - [glob document](https://docs.python.org/3/library/glob.html) | [shutil document](https://docs.python.org/3/library/shutil.html)

    ※ 해당 모듈 이외에 자신이 잘 알고 있는 방법을 사용해도 됩니다.
    
---

In [ ]:
########################
# 이 셀부터 코드 작성하세요
########################
# 3. 데이터를 Training set | Validation set으로 분할하세요.
from sklearn.model_selection import train_test_split
dst_train_img = '/content/Dataset/images/train'
dst_val_img = '/content/Dataset/images/val'
dst_train_label = '/content/Dataset/labels/train'
dst_val_label = '/content/Dataset/labels/val'
for won in won_list:

    image_dir = os.path.join(data_path, won)
    label_dir = os.path.join(data_path, won)

    # shutil를 사용하기 위해서 경로까지 가져오는 glob을 사용한다.
    file_list_img = glob.glob(os.path.join(image_dir, '*.jpg'))
    file_list_label = glob.glob(os.path.join(label_dir, '*.json'))

    file_list_img.sort()
    file_list_label.sort()

    print('='*30)
    print(won)
    print(len(file_list_img), len(file_list_label))
    train_jpg, val_jpg, train_json, val_json = train_test_split(file_list_img, file_list_label, test_size=0.2)
    print(len(train_jpg), len(val_jpg), len(train_json), len(val_json))

    # .copy(소스경로, 대상경로) : 소스경로의 파일을 대상경로로 복사
    for jpg in train_jpg:
        shutil.copy(jpg, dst_train_img)
    for jpg in val_jpg:
        shutil.copy(jpg, dst_val_img)
    for json in train_json:
        shutil.copy(json, dst_train_label)
    for json in val_json:
        shutil.copy(json, dst_val_label)

10
436 436
348 88 348 88
50
440 440
352 88 352 88
100
440 440
352 88 352 88
500
440 440
352 88 352 88
1000
858 858
686 172 686 172
5000
867 867
693 174 693 174
10000
867 867
693 174 693 174
50000
870 870
696 174 696 174


### (2) json에서 정보 추출
---
- **세부요구사항**
    - json 파일에서 필요한 정보를 추출하세요:
        - 위치 정보 : x1, x2, y1, y2
        - 박스 정보 : shape_type
        - 클래스 정보 : labels
    - 화폐당 하나의 클래스로 변경하세요.
        - json 파일에는 화폐 클래스가 앞뒷면으로 구분되어 있습니다.
        - 화폐의 앞뒷면 구분을 없애주세요.
            - 예시 : 'ten_front', 'ten_back' -> 'ten'
    - 화폐의 위치 정보를 YOLO 모델 형식에 맞게 변경 해주세요.
        - 사용되는 이미지는 원본에서 1/5로 축소되어 있습니다.
        - json 파일의 정보는 원본 기준 데이터이므로 위치 정보 추출을 할 때 x값과 y값을 1/5로 줄여주세요.
    - 이렇게 변경된 정보를 YOLO label 형식에 맞게 txt파일로 저장 해 주세요.
        - Hint : YOLO Labeling Format [label, x-center, y-center, width-norm, height-norm]
---

In [ ]:
import os, json

In [ ]:
json_path = '/content/Dataset/labels/'
temp_list = ['train', 'val']

In [ ]:
!mkdir /content/Dataset/jstotxt;
!mkdir /content/Dataset/jstotxt/train;
!mkdir /content/Dataset/jstotxt/val;

In [ ]:
########################
# 이 셀부터 코드 작성하세요
# Json 파일에서 필요한 정보만 골라 txt로 바꾸는 작업임을 기억하세요!
########################

for temp in temp_list:
    path = os.path.join(json_path, temp)
    files_json = os.listdir(path)
    #print(len(file_json))

    # 텍스트 파일 저장 경로 지정
    txt_path = '/content/Dataset/jstotxt'
    txt_all_path = os.path.join(txt_path, temp)

    for js_file in files_json:
        js_path = os.path.join(path, js_file)
        with open(js_path, 'r') as f:
            js = json.load(f)
        label = js['imagePath'].split('_', 1)[0]
        #  {0:'10', 1:'50', 2:'100', 3:'500', 4:'1000', 5:'5000', 6:'10000', 7:'50000'}
        # label_dic = {10:0, 50:1, 100:2, 500:3, 1000:4, 5000:5, 10000:6, 50000:7}
        # label = label_dic[label]
        if label == '10':
            label = 0
        elif label == '50':
            label = 1
        elif label == '100':
            label = 2
        elif label == '500':
            label = 3
        elif label == '1000':
            label = 4
        elif label == '5000':
            label = 5
        elif label == '10000':
            label = 6
        elif label == '50000':
            label = 7

        x1 = js['shapes'][0]['points'][0][0]/5
        y1 = js['shapes'][0]['points'][0][1]/5
        x2 = js['shapes'][0]['points'][1][0]/5
        y2 = js['shapes'][0]['points'][1][1]/5

        x_center = (x1+x2)/2
        y_center = (y1+y2)/2

        org_width = js['imageWidth']/5
        org_height = js['imageHeight']/5

        width = x2-x1
        height = y2-y1

        # 텍스트 파일 저장 경로와 파일명 까진 합친 경로
        tpath = os.path.join(txt_all_path, js_file.replace('.json', '.txt'))
        with open(tpath, 'w') as txt_file:
            txt_file.write(f'{label} {x_center/org_width} {y_center/org_height} {width/org_width} {height/org_height}')

In [ ]:
# 폴더 삭제
!rm -r /content/Dataset/labels/train*
!rm -r /content/Dataset/labels/val*
# 폴더 이동
shutil.move('/content/Dataset/jstotxt/train', '/content/Dataset/labels')
shutil.move('/content/Dataset/jstotxt/val', '/content/Dataset/labels')

'/content/Dataset/labels/val'

### (3) 데이터셋 정보가 담긴 파일 생성
---
- **세부요구사항**
    - 파일 안에 있어야 할 정보는 아래와 같습니다.
        - 학습할 클래스 이름 정보
        - 학습할 클래스 수 정보
        - Training, Validation 데이터셋 위치 정보
---
- 가장 대중적으로 이용하는 라이브러리는 yaml 입니다.
    - [yaml document](https://pyyaml.org/wiki/PyYAMLDocumentation)
    - 해당 모듈 이외에 자신이 잘 알고 있는 방법을 사용해도 됩니다.
---

In [ ]:
import yaml

In [ ]:
won_dict = {0:'10', 1:'50', 2:'100', 3:'500', 4:'1000', 5:'5000', 6:'10000', 7:'50000'}

In [ ]:
########################
# 이 셀부터 코드 작성하세요
########################
data = {
    'names': won_dict,
    'nc': 8,
    'path' : '/content/Dataset',
    'train': 'images/train',
    'val': 'images/val'
}
with open('/content/Dataset/money.yaml', 'w') as f :
    yaml.dump(data, f, default_flow_style=False)

## 3.모델링

### (1) 모델 라이브러리 설치
---

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone
!pip install -r yolov5/requirements.txt  # install

Cloning into 'yolov5'...
remote: Enumerating objects: 15994, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 15994 (delta 18), reused 18 (delta 12), pack-reused 15967
Receiving objects: 100% (15994/15994), 14.58 MiB | 7.08 MiB/s, done.
Resolving deltas: 100% (10984/10984), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.0/190.0 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 618.4/618.4 kB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.0 MB/s eta 0:00:00


### (2) 가중치 파일 다운로드
---
- **세부요구사항**
    - 모델 개발자가 제공하는 사전 학습 가중치 파일을 다운로드 하세요.
        - 해당 과정이 불필요하다면 넘어가셔도 됩니다!
---

In [ ]:
########################
# 이 셀부터 코드 작성하세요
########################
/content/Dataset/money.yaml

### (3) 학습 : train.py
---
- **세부요구사항**
    - UltraLytics YOLO v5에는 아래의 데이터가 필요합니다.
        - 데이터셋 정보가 담긴 yaml 파일
        - 사용하려는 모델 구조에 대한 yaml 파일
        - 사용하려는 모델의 가중치 파일
---

In [ ]:
########################
# 이 셀부터 코드 작성하세요
########################
# !cd python train.py --img 640 --epochs 3 --data coco128.yaml --weights yolov5s.pt

In [ ]:
!pip install ultralytics

In [ ]:
from ultralytics import YOLO

In [ ]:
model = YOLO(model = 'yolov5n.yaml', task='detect')

In [ ]:
model.train(data='/content/Dataset/money.yaml',
            epochs=10,
            pretrained=True,
            verbose=True,
            )

In [ ]:
model.predict(source='/content/KakaoTalk_20230922_105144925.png',
              save=True,
              conf=0.1,
              iou=0.5,
              line_width=2
              )

In [ ]:
model.predict(source='/content/Dataset/50000/50000_B_DESK_0_102.jpg',
              save=True,
              conf=0.25,
              iou=0.7,
              line_width=2
              )

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!python /content/yolov5/train.py --img 640 --batch 64 --epochs 50 --data /content/Dataset/money.yaml --weights yolov5s.pt

train: weights=yolov5s.pt, cfg=, data=/content/Dataset/money.yaml, hyp=yolov5/data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=64, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5/runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-224-g6262c7f Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gam

## 4.탐지 : detect.py
---
- **세부요구사항**
    - 학습 과정에서 생성된 가중치 파일을 이용하세요.
    - IoU threshold를 0.25 이하로 설정하세요.
    - confidence threshold를 0.75 이상으로 설정하세요.
---
- 여러분이 **직접 촬영한 화폐 사진과 동영상**을 탐지 과정에 이용하여 결과를 확인하세요.
    - 조건
        1. 화폐의 수를 늘려가며 촬영 해보세요.
            - ex) 50원 하나, 50원 둘, 50원 셋, ...
        2. 화폐의 종류를 늘려가며 촬영 해보세요.
            - ex) 50원 하나와 100원 하나, 50원 하나와 100원 하나와 1000원 하나, ...
        3. 사진은 최소 30장 이상, 동영상은 최소 하나 이상 촬영하여 사용 해보세요.
---

In [ ]:
!python /content/yolov5/detect.py --weights /content/yolov5/runs/train/exp2/weights/best.pt --source /content/drive/MyDrive/Mini_Project3/mini3_test_set_2team --conf 0.75 --iou 0.25 --imgsz 640;

detect: weights=['/content/yolov5/runs/train/exp2/weights/best.pt'], source=/content/drive/MyDrive/Mini_Project3/mini3_test_set_2team, data=yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.75, iou_thres=0.25, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-224-g6262c7f Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)

Fusing layers... 
Model summary: 157 layers, 7031701 parameters, 0 gradients, 15.8 GFLOPs
image 1/45 /content/drive/MyDrive/Mini_Project3/mini3_test_set_2team/001.jpeg: 480x640 1 100, 2 500s, 84.2ms
image 2/45 /content/drive/MyDrive/Mini_Project3/mini3_test_set_2team/002.jpeg: 480x640 1 1000, 1 10000, 1 50000, 6.9ms
image 3/45 /content/

In [ ]:
!cp -r /content/yolov5/runs/detect/exp2 /content/drive/MyDrive/Mini_Project3

In [ ]:
!python /content/yolov5/detect.py --weights /content/yolov5/runs/train/exp2/weights/best.pt --source /content/drive/MyDrive/Mini_Project3/mini3_test_set_2team --conf 0.25 --iou 0.75 --imgsz 640;

detect: weights=['/content/yolov5/runs/train/exp2/weights/best.pt'], source=/content/drive/MyDrive/Mini_Project3/mini3_test_set_2team, data=yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.75, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-224-g6262c7f Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)

Fusing layers... 
Model summary: 157 layers, 7031701 parameters, 0 gradients, 15.8 GFLOPs
image 1/45 /content/drive/MyDrive/Mini_Project3/mini3_test_set_2team/001.jpeg: 480x640 2 100s, 2 500s, 86.4ms
image 2/45 /content/drive/MyDrive/Mini_Project3/mini3_test_set_2team/002.jpeg: 480x640 2 1000s, 1 10000, 1 50000, 6.6ms
image 3/45 /conten

In [ ]:
!cp -r /content/yolov5/runs/detect/exp3 /content/drive/MyDrive/Mini_Project3

In [ ]:
print(1)

1
